# The World Factbook

In this an introductory project, we will familiarize ourselves with the basic workflow of SQL and run some analytical queries.

<img src="https://images.unsplash.com/photo-1651421479704-470a78eef530?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=870&q=80" width=800 height=200/>
Photo by <a href="https://unsplash.com/@luisdesiro?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Luis Desiro</a> on <a href="https://unsplash.com/s/photos/countries-flags?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>


## Project and Data Overview

We'll work with the data from [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

- `population` - the globle population.
- `population_growth` - the annual population growth rate, as a percentage.
- `area` - the total land and water area.

In this project, we'll use SQL to analyze data from this database. We'll use the following code to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

We get no error. Let's run some queries to test if the database is connected properly.

Let's query the database to get information about table.

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


The database actually contains only one table name 'facts'.

To run SQL queries in this project, we add `%%sql` on its own line to the start of our query. So to run our query that returns the first five rows of the `facts` table in the database will look like this:

In [3]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;  

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Executing the query above, we see the first few rows of our `facts` table. Here are the description for some of the columns:

- `name` - the name of the country.
- `area` - the country's total area (both land and water).
- `area_land` - the country's land area in square kilometers.
- `area_water` - the country's waterarea in square kilometers.
- `population` - the country's population.
- `population_growth` - the country's population growth as a percentage.
- `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` - the country's death rate, or the number of death per year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

## Summary Statistics

We'll write a query that returns the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [4]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics after running the above query:

- There's a country with a population of `0`
- There's a country with a population of `7256490011` (or more than 7.2 billion people)

Let's use the subqueries to zoom in on just these countries *without* using the specific values.

## Exploring Outliers

We'll write two queries for finding the country with a minimum population, and also finding the country with a maximum population.

We begin with the country that has the minimum population.

In [11]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MIN(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0


It seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/):

<img src="https://s3.amazonaws.com/dq-content/257/fb_antarctica.png" width=800 height=800/>

Next we'll find out the country with the maximum population.

In [12]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


name,population
World,7256490011


It seems like the table also contains a row for the whole world, which explains the population of over 7.2 billion. But we are only interested in the country's population, not in the whole world population.

## Summary Statistics Revisited

Now that we know this, we should recalculate the summary statistics we calculated earlier, this time excluding the row for the whole world.

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


There is a country whose population closes in on 1.4 billion!

## Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet (i.e. `World`).

In [28]:
%%sql
SELECT ROUND(AVG(population), 2) AS avg_pop,
       ROUND(AVG(area), 2) AS avg_area
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
62094928.32,555093.55


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

## Finding Densely Populated Countries

In the next step, we'll build on the query we wrote above to find countries that are densely populated. We'll identify countries that have the following:

- Above-average values for population.
- Below-average values for area.

In [31]:
%%sql
SELECT name, population, area
    FROM facts
  Where population > (SELECT AVG(population)
                         FROM facts)
   AND area < (SELECT AVG(area)
               FROM facts)
  ORDER BY population DESC;

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Japan,126919659,377915
Philippines,100998376,300000
Vietnam,94348835,331210
Germany,80854408,357022
Thailand,67976405,513120
United Kingdom,64088222,243610
